In [1]:
import pandas as pd

### Read data

In [84]:
df = pd.read_csv('data-related/data.csv')

### Preprocess data

In [86]:
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.rstrip('_')

In [87]:
df['Приоритет'] = df.groupby(['Подразделение_1', 'Функциональный_блок', 'Должность', 'Роль']).cumcount() + 1

In [88]:
df['Приоритет'] = df['Приоритет'].fillna(0).astype(int)

In [95]:
df

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Телефон,Город,Адрес,Почта,Приоритет
0,Центральный офис,NaN,NaN,NaN,NaN,Председатель банка,руководство,Смирнов,Александр,79699822593,Москва,"проспект Мира, 14, 3 этаж, офис 301",NaN,0
1,Центральный офис,Корпоративный блок,NaN,NaN,NaN,Заместитель председателя банка,руководство,Иванов,Алексей,79227639290,Москва,"проспект Мира, 14, 3 этаж, офис 302",NaN,1
2,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,NaN,NaN,Владелец продукта,руководство,Кузнецов,Альберт,79182913214,Москва,"проспект Мира, 14, 2 этаж, место 1",NaN,1
3,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,команда 1,NaN,UX/UI Дизайнер,Дизайнер,Соколов,Александр,79700231854,Москва,"проспект Мира, 14, 2 этаж, место 2",NaN,1
4,Центральный офис,Корпоративный блок,онлайн-банкинг для бизнеса,команда 1,NaN,UX Дизайнер,Дизайнер,Попов,Алексей,79254156549,Москва,"проспект Мира, 14, 2 этаж, место 3",NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Руководитель офиса,руководство,Соловьёва,Акулина,79833706058,Волгоград,улица Татищева 27,NaN,4
210,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Менеджер по продажам,продажи,Васильева,Алевтина,79681314418,Волгоград,улица Татищева 27,NaN,4
211,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Ведущий менеджер по продажам,продажи,Зайцева,Александра,79515700252,Волгоград,улица Татищева 27,NaN,4
212,Филиал в Волгоградской области,Розничный блок,NaN,NaN,Дополнительный офис 4,Кассир,обслуживание,Павлов,Алексей,79997618652,Волгоград,улица Татищева 27,NaN,4


### Recsys case

### BD search script

In [111]:
def searchEmployees(df, **kwargs):
    mask = pd.Series([True] * len(df))

    for key, value in kwargs.items():
        match key:
            case 'name':
                mask = mask & (df['Имя'] == value)
            case 'surname':
                mask = mask & (df['Фамилия'] == value)
            case 'full_name':
                full_name_parts = value.split()
                if len(full_name_parts) == 2:
                    first_name, last_name = full_name_parts
                    mask = mask & (df['Имя'] == first_name) & (df['Фамилия'] == last_name)
            case 'position':
                mask = mask & (df['Должность'] == value)
            case 'department_1':
                mask = mask & (df['Подразделение_1'] == value)
            case 'department_2':
                mask = mask & (df['Подразделение_2'] == value)
            case 'department_3':
                mask = mask & (df['Подразделение_3'] == value)
            case 'department_4':
                mask = mask & (df['Подразделение_4'] == value)
            case 'functional_block':
                mask = mask & (df['Функциональный_блок'] == value)
            case 'role':
                mask = mask & (df['Роль'] == value)

    result = df[mask]
    return result

In [112]:
searchEmployees(df, name='Иван')

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Телефон,Город,Адрес,Почта,Приоритет


In [113]:
searchEmployees(df, position='Разработчик')

,Подразделение_1,Функциональный_блок,Подразделение_2,Подразделение_3,Подразделение_4,Должность,Роль,Фамилия,Имя,Телефон,Город,Адрес,Почта,Приоритет
